In [1]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
import numpy as np
import matplotlib.pyplot as plt
from pymoo.indicators.hv import HV
from pymoo.problems import get_problem
from pymoo.util.dominator import Dominator
import cma
import json
from ipynb.fs.full.nds import fast_non_dominated_sort
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [2]:
def calculate_hv(PF,r):
#     hv = get_performance_indicator("hv", ref_point=r)
    ind = HV(ref_point=r)
    HV_PF = ind(PF)
    return HV_PF


#GP - modelling

In [3]:
def GP(X,Y):
    no_var = X.shape[1]
    l0 = 2*np.random.rand(1,no_var)
    l0 = l0.flatten()
    #k = gpflow.kernels.Matern52(lengthscales=l0)
    k = gpflow.kernels.SquaredExponential(lengthscales=l0)
    #k = gpflow.kernels.Matern52()
    m = gpflow.models.GPR(data=(X, Y), kernel=k, mean_function=None)
    opt = gpflow.optimizers.Scipy()
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=1000),method = 'BFGS')
    return m, m.log_marginal_likelihood()

In [4]:
def EHVI_2D(PF,r,mu,sigma):
    n = PF.shape[0]
    S1 = np.array([r[0],-np.inf])
    S1 = S1.reshape(1,-1)
    Send = np.array([-np.inf,r[1]])
    Send = Send.reshape(1,-1)
    index = np.argsort(PF[:,1])
    
    S = PF[index,:]
    
    S = np.concatenate((S1,S,Send),axis = 0)
    
    y1 = S[:,0] 
    y2 = S[:,1]
    
    y1 = y1.reshape(-1,1)
    y2 = y2.reshape(-1,1)
    
    mu = mu.reshape(1,-1)
    sigma = sigma.reshape(1,-1)
        
    sum_total1 = 0;
    sum_total2 = 0;
    
    for i in range(1,n+2):
        t = (y1[i] - mu[0][0])/sigma[0][0]
        if i==n+1:
            sum_total1 = sum_total1
        else:
            sum_total1 = sum_total1 + (y1[i-1] - y1[i])*stats.norm.cdf(t)*psi_cal(y2[i],y2[i],mu[0][1],sigma[0][1])
        sum_total2 = sum_total2 + (psi_cal(y1[i-1],y1[i-1],mu[0][0],sigma[0][0]) \
                                   - psi_cal(y1[i-1],y1[i],mu[0][0],sigma[0][0]))*psi_cal(y2[i],y2[i],mu[0][1],sigma[0][1])
        
    EHVI = sum_total1 + sum_total2
    return EHVI


def psi_cal(a,b,m,s):
    t = (b - m)/s
    return s*stats.norm.pdf(t) + (a - m)*stats.norm.cdf(t)

In [5]:
def EHVI_monte_carlo(PF,r,mu,sigma,HV_PF):
    mu = mu.reshape(-1)
    sigma = sigma.reshape(-1)
    Sigma = np.diag(sigma**2)
    samples = np.random.multivariate_normal(mean = mu,cov = Sigma,size=50000)
    n_samples = samples.shape[0]
    HV_cont = np.zeros(n_samples);
    for i in range(0,n_samples):
        #print(i)
        s = samples[i,:]
        s = s.reshape(1,-1)
        PF_temp = np.concatenate((PF,s),axis=0)
        #print(PF_temp)
        front_index = fast_non_dominated_sort(PF_temp)
        #print(front_index[0])
        front_index = np.asarray(front_index[0])
        #print(front_index)
        PF_i = PF_temp[front_index,:]
        PF_i = PF_i.reshape(-1,F.shape[1])
        HV_cont[i] = calculate_hv(PF_i,r) - HV_PF

    return np.mean(HV_cont)

In [6]:
def AF(x,thisdict):
    
    #thisdict = {"model1": m1, "model2": m2,"Pareto_front": PF,"HV_PF": HV_PF,"ref_point": r}
    
#     print(thisdict)
    
    m1 = thisdict["model1"]
    m2 = thisdict["model2"]
    
    PF = thisdict["Pareto_front"]
    HV_PF = thisdict["HV_PF"]
    r = thisdict["ref_point"]
    
    
#     print(m1,PF)

#     print(x,x.shape)
    
    x = x.reshape(1,-1)
    
    [y_pred1,sigma1] = m1.predict(x,return_std=True)
    [y_pred2,sigma2] = m2.predict(x,return_std=True)
    
#     print(y_pred1,sigma1)

#     mu = np.concatenate((y_pred1,y_pred2),axis=1)
#     sigma_vector = np.concatenate((sigma1,sigma2),axis=1)
#     sigma = np.sqrt(var_vector)
    
    mu = np.transpose(np.vstack([y_pred1,y_pred2]))
    sigma = np.transpose(np.vstack([sigma1,sigma2]))
        
    
    
#     return -EHVI_monte_carlo(PF,r,mu,sigma,HV_PF)

    return -EHVI_2D(PF,r,mu,sigma)
    
    


In [ ]:
no_var = 2
problem = get_problem('dtlz2',n_var=no_var,n_obj=2)
lb = np.zeros(no_var)
ub = np.ones(no_var)
X = lb + (ub - lb)*np.random.rand(10,no_var)
F = problem.evaluate(X) 
r = np.max(F,0)+ 2

lb = lb.reshape(1,-1)
ub = ub.reshape(1,-1)


# main loop
for i in range(0,1):
    # non dominated sorting
    front_index = fast_non_dominated_sort(F)
    front_index = np.asarray(front_index[0])
    PF = F[front_index,:]
    PF = PF.reshape(-1,F.shape[1])
    
    HV_PF = calculate_hv(PF,r)
    
    kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0),nu=2.5)
    m1 = GaussianProcessRegressor(kernel=kernel,alpha=1e-5,n_restarts_optimizer=10,normalize_y=True,optimizer='fmin_l_bfgs_b').fit(X, F[:,0])
    m2 = GaussianProcessRegressor(kernel=kernel,alpha=1e-5,n_restarts_optimizer=10,normalize_y=True,optimizer='fmin_l_bfgs_b').fit(X, F[:,1])
    
    
#     m1,ml1 = GP(X,F[:,0])
#     m2,ml2 = GP(X,F[:,1])
    
#     print(type(m1))
    

    x0 = np.random.rand(1,no_var)
    x0 = x0.flatten()
    opts = cma.CMAOptions()
    #opts.set("bounds", [lb, ub])
    
    thisdict = {"model1": m1, "model2": m2,"Pareto_front": PF,"HV_PF": HV_PF,"ref_point": r} 
    #res = cma.fmin(maximise_AF, x0, 0.5, args=('m1','m2','PF','HV_PF','r',),restarts=0)
    opts = cma.CMAOptions()
    opts.set("bounds", [[0, 0], [1, 1]])
    xopt,es = cma.fmin2(AF, x0, 0.5, opts,args=(thisdict,),restarts=1)
    print(xopt)
    es.plot()

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=510158, Fri Sep 22 12:13:27 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 -2.152424978281917e-01 1.0e+00 4.58e-01  4e-01  4e-01 0:00.0
    2     12 -1.388682329316197e-01 1.1e+00 5.54e-01  5e-01  6e-01 0:00.1
    3     18 -3.793571458321085e-01 1.5e+00 5.17e-01  5e-01  5e-01 0:00.1
   72    432 -4.131872159122364e-01 3.4e+00 3.29e-05  5e-08  1e-07 0:03.2
   76    456 -4.131872159122102e-01 3.0e+00 2.15e-05  2e-08  8e-08 0:03.3
termination on tolfun=1e-11 (Fri Sep 22 12:13:31 2023)
final/bestever f-value = -4.131872e-01 -4.131872e-01
incumbent solution: [5.04775381788508e-16, 0.4213937048063849]
std deviation: [2.0524999456872108e-08, 7.812209906654941e-08]
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=510159, Fri Sep 22 12:13:31 2023)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1    469 -2.514468508130510e-01 1.0e+00 5.59e-01  6e-01  6e-01 0:00